In [28]:
import pandas as pd
from sqlalchemy import MetaData, Table, create_engine, select, and_, or_, not_, text
from sqlalchemy.sql import func, distinct, exists

In [29]:
engine = create_engine('sqlite:///../Data/Students.db')
metadata_obj = MetaData() # Объект-контейнер, содержит информацию о схеме данных
metadata_obj.reflect(bind=engine) # Отображение БД

In [30]:
exam_marks=Table("exam_marks", metadata_obj, autoload=True, autoload_with="engine")
lecturer=Table("lecturer", metadata_obj, autoload=True, autoload_with="engine")
student=Table("student", metadata_obj, autoload=True, autoload_with="engine")
subj_lect=Table("subj_lect", metadata_obj, autoload=True, autoload_with="engine")
subject=Table("subject", metadata_obj, autoload=True, autoload_with="engine")
university=Table("university", metadata_obj, autolad=True, autolad_with="engine")

In [31]:
conn=engine.connect() # Соединение

<img src=../Img/pencil.png style="width:190px;height:190px">
<p style="background-color:powderblue; font-size:125%; color: rebeccapurple;">Выполните задания</p>
<p>Выполните задания с помощью соедиинения таблиц в Jupyter Notebook </p>

1. Составьте список названий университетов с указанием суммарного количества аудиторных часов на изучение каждого предмета.

2. Получите список названий университетов вместе с фамилиями и именами студентов, получающих максимальную для каждого университета стипендию.

3. Напишите запрос для получения списка преподавателей, преподающих только один предмет. Отсортируйте результат по фамилии преподавателя. Результат должен содержать имя и фамилию преподавателя, название предмета.

4. Напишите запрос для получения списка университетов вместе с фамилиями студентов, получивших хотя бы одну отличную оценку. Результат отсортируйте по названию университета.

5. Напишите запрос, выполняющий вывод имен и фамилий преподавателей, проводящих занятия на первом курсе. Отсортируйте результат по фаимилии преподавателя.

6. Напишите запрос, выполняющий вывод наименований предметов обучения, читаемых двумя или более преподавателями. Результат должен состоять из одного столбца названий предметов, без повторов, в алфавитном порядке по возрастанию.

# Решение

In [32]:
#1. Составьте список названий университетов с указанием суммарного количества аудиторных часов на изучение каждого предмета.

# SQL - SELECT univ_name, SUM(hour) FROM university JOIN lecturer l on university.univ_id = l.univ_id JOIN subj_lect sl on l.lecturer_id = sl.lecturer_id JOIN subject s on sl.subj_id = s.subj_id GROUP BY l.univ_id

s = select(university.c.univ_name, func.sum(subject.c.hour).label("Кол-во часов")).join(lecturer, university.c.univ_id == lecturer.c.univ_id).join(subj_lect, lecturer.c.lecturer_id == subj_lect.c.lecturer_id).join(subject, subj_lect.c.subj_id == subject.c.subj_id).group_by(university.c.univ_id)

print(s)
pd.read_sql(s, conn)

SELECT university.univ_name, sum(subject.hour) AS "Кол-во часов" 
FROM university JOIN lecturer ON university.univ_id = lecturer.univ_id JOIN subj_lect ON lecturer.lecturer_id = subj_lect.lecturer_id JOIN subject ON subj_lect.subj_id = subject.subj_id GROUP BY university.univ_id


,univ_name,Кол-во часов
0,ВГУ,284
1,НГУ,34
2,ТГУ,146
3,МГУ,146
4,РГУ,90
5,МГТУ им. Баумана,90
6,"ФГБОУ ""Политех""",90
7,КФУ,146
8,УЛГУ,34


In [33]:
#2. Получите список названий университетов вместе с фамилиями и именами студентов, получающих максимальную для каждого университета стипендию.

# SQL - SELECT univ_name, surname, name, MAX(stipend) FROM student JOIN university u on u.univ_id = student.univ_id GROUP BY student.univ_id

s = select(university.c.univ_name, student.c.surname, student.c.name, func.max(student.c.stipend).label("Стипендия")).join(university, university.c.univ_id == student.c.univ_id).group_by(student.c.univ_id)

print(s)
pd.read_sql(s, conn)

SELECT university.univ_name, student.surname, student.name, max(student.stipend) AS "Стипендия" 
FROM student JOIN university ON university.univ_id = student.univ_id GROUP BY student.univ_id


,univ_name,surname,name,Стипендия
0,ВГУ,Зайцева,Ольга,250
1,НГУ,Трофимов,Владислав,250
2,БГУ,Суханов,Константин,250
3,ТГУ,Мельникова,Светлана,250
4,ВГМА,Кондрат,Синицин,200
5,МГУ,Блохина,Ираида,250
6,РГУ,Панов,Арсений,250
7,ФинУ,Казакова,Ольга,250
8,МГТУ им. Баумана,Миронова,Федор,200
9,"ФГБОУ ""Политех""",Миронова,Анна,200


In [34]:
# 3. Напишите запрос для получения списка преподавателей, преподающих только один предмет. Отсортируйте результат по фамилии преподавателя. Результат должен содержать имя и фамилию преподавателя, название предмета.

# SQL - SELECT name, surname, subj_name, COUNT(sl.subj_id) FROM lecturer JOIN subj_lect sl on lecturer.lecturer_id = sl.lecturer_id JOIN subject s on s.subj_id = sl.subj_id GROUP BY lecturer.surname HAVING COUNT(sl.subj_id) == 1

s = select(lecturer.c.name, lecturer.c.surname, subject.c.subj_name, func.count(subj_lect.c.subj_id).label("Преподает предметов")).join(subj_lect, lecturer.c.lecturer_id == subj_lect.c.lecturer_id).join(subject, subject.c.subj_id == subj_lect.c.subj_id).group_by(lecturer.c.surname).having(func.count(subj_lect.c.subj_id) == 1)

print(s)
pd.read_sql(s, conn)

SELECT lecturer.name, lecturer.surname, subject.subj_name, count(subj_lect.subj_id) AS "Преподает предметов" 
FROM lecturer JOIN subj_lect ON lecturer.lecturer_id = subj_lect.lecturer_id JOIN subject ON subject.subj_id = subj_lect.subj_id GROUP BY lecturer.surname 
HAVING count(subj_lect.subj_id) = :count_1


,name,surname,subj_name,Преподает предметов
0,Елисей,Алексеев,ОБЖ,1
1,Мефодий,Беляков,Информатика,1
2,Нинель,Брагина,ОБЖ,1
3,Прокофий,Дорофеев,Программирование на языках Python и SQL,1
4,Юлиан,Зимин,Программирование на языках Python и SQL,1
5,Борис,Колесников,Физика,1
6,Анна,Кудрявцева,Информатика,1
7,Павел,Лагутин,Английский язык,1
8,Акулина,Мамонтова,Программирование на языках Python и SQL,1
9,Иван,Никонов,История,1


In [35]:
# 4. Напишите запрос для получения списка университетов вместе с фамилиями студентов, получивших хотя бы одну отличную оценку. Результат отсортируйте по названию университета.

s = select(student.c.student_id).join(exam_marks, exam_marks.c.student_id == student.c.student_id).where(exam_marks.c.mark == 5)

s_2 = select(university.c.univ_name, student.c.surname).join(student, student.c.univ_id == university.c.univ_id).where(student.c.student_id.in_(s)).order_by(university.c.univ_name)

print(s_2)
pd.read_sql(s_2, conn)

SELECT university.univ_name, student.surname 
FROM university JOIN student ON student.univ_id = university.univ_id 
WHERE student.student_id IN (SELECT student.student_id 
FROM student JOIN exam_marks ON exam_marks.student_id = student.student_id 
WHERE exam_marks.mark = :mark_1) ORDER BY university.univ_name


,univ_name,surname
0,ВГУ,Кузнецов
1,ВГУ,Зайцева
2,ВГУ,Белкин
3,ВГУ,Жданова
4,ВГУ,Борисова
5,МГТУ им. Баумана,Лапина
6,МГУ,Блохина
7,ТГУ,Пак
8,ТГУ,Мельникова
9,"ФГБОУ ""Политех""",Васильев


In [36]:
# 5. Напишите запрос, выполняющий вывод имен и фамилий преподавателей, проводящих занятия на первом курсе. Отсортируйте результат по фаимилии преподавателя.

# SQL - SELECT lecturer.name, lecturer.surname FROM lecturer JOIN student s on lecturer.univ_id = s.univ_id WHERE kurs == 1 GROUP BY lecturer.surname

s = select(lecturer.c.name, lecturer.c.surname).join(student, lecturer.c.univ_id == student.c.univ_id).where(student.c.kurs == 1).group_by(lecturer.c.surname)

print(s)
pd.read_sql(s, conn)

SELECT lecturer.name, lecturer.surname 
FROM lecturer JOIN student ON lecturer.univ_id = student.univ_id 
WHERE student.kurs = :kurs_1 GROUP BY lecturer.surname


,name,surname
0,Елисей,Алексеев
1,Майя,Артемьева
2,Мефодий,Беляков
3,Юлиан,Беспалов
4,Оксана,Богданова
5,Михаил,Волков
6,Прокофий,Дорофеев
7,Юлия,Зайцева
8,Юлиан,Зимин
9,Оксана,Зыкова


In [37]:
# 6. Напишите запрос, выполняющий вывод наименований предметов обучения, читаемых двумя или более преподавателями. Результат должен состоять из одного столбца названий предметов, без повторов, в алфавитном порядке по возрастанию.

# SQL - SELECT subj_name FROM subject JOIN subj_lect sl on subject.subj_id = sl.subj_id JOIN lecturer l on l.lecturer_id = sl.lecturer_id GROUP BY subj_name HAVING COUNT(l.lecturer_id) >= 2

s = select(subject.c.subj_name.label("Название предмета")).join(subj_lect, subject.c.subj_id == subj_lect.c.subj_id).join(lecturer, lecturer.c.lecturer_id == subj_lect.c.lecturer_id).group_by(subject.c.subj_name).having(func.count(lecturer.c.lecturer_id) >= 2)

print(s)
pd.read_sql(s, conn)

SELECT subject.subj_name AS "Название предмета" 
FROM subject JOIN subj_lect ON subject.subj_id = subj_lect.subj_id JOIN lecturer ON lecturer.lecturer_id = subj_lect.lecturer_id GROUP BY subject.subj_name 
HAVING count(lecturer.lecturer_id) >= :count_1


,Название предмета
0,Английский язык
1,Информатика
2,История
3,ОБЖ
4,Программирование на языках Python и SQL
